In [137]:
# Import packages
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
from mlxtend.regressor import StackingCVRegressor
from catboost import Pool, cv, CatBoostRegressor
from scipy.special import boxcox1p
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, Lasso, RidgeCV, Ridge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [138]:
# Load data
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

train_data['Training'] = 1
test_data['Training'] = 0
numerical_features = train_data.dtypes[train_data.dtypes != 'object'].index.values
categorical_features = train_data.dtypes[train_data.dtypes == 'object'].index.values
submit = ['SalePrice', 'Id']
significant_columns_cat = ['Neighborhood', 'ExterQual', 'BsmtQual', 'KitchenQual', 'GarageFinish',
 'FireplaceQu', 'Foundation', 'GarageType', 'BsmtFinType1', 'HeatingQC',
 'MasVnrType', 'BsmtExposure', 'SaleCondition', 'Exterior1st', 'Exterior2nd',
 'SaleType', 'MSZoning', 'HouseStyle', 'GarageQual', 'GarageCond']

In [139]:
# preprocessing
# remove outliers
# LotFrontage > 300
train_data.drop(train_data[train_data['LotFrontage']>300].index)
# LotArea > 100000
train_data.drop(train_data[train_data['LotArea']>100000].index)
# BsmtFinSF1 > 4000
train_data.drop(train_data[train_data['BsmtFinSF1']>4000].index)
# GrLivArea > 4000
train_data.drop(train_data[train_data['GrLivArea']>4500].index)
# TotalBsmtSF > 5000
train_data.drop(train_data[train_data['TotalBsmtSF']>5000].index)
# OpenPorchSF > 500
train_data.drop(train_data[train_data['OpenPorchSF']>500].index)
# 1stFlrSF > 4000
train_data.drop(train_data[train_data['1stFlrSF']>4000].index)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Training
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,8,2007,WD,Normal,175000,1
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000,1
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500,1
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2010,WD,Normal,142125,1


In [140]:
# concat training and testing
y = train_data['SalePrice']
all_data = pd.concat([train_data, test_data], ignore_index=True)

# fill categoricals
categoricals = all_data[categorical_features]
categoricals['MSZoning'] = categoricals['MSZoning'].fillna(categoricals['MSZoning'].mode()[0])
categoricals.fillna('None', inplace=True)
all_data[categorical_features] = categoricals

# fill in numeric
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].apply(lambda x: x.fillna(x.median()))
all_data.fillna(0, inplace=True)

# select significant columns
significant_columns = [*significant_columns_cat, *numerical_features]
all_data = all_data[significant_columns]

In [141]:
# feature engineering
# convert non-categorical to categorical
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].apply(str)

# add in more features
all_data['TotalSF'] = all_data['GrLivArea'] + all_data['TotalBsmtSF']
all_data['Bathrooms'] = all_data['BsmtFullBath'] + all_data['FullBath'] + (all_data['BsmtHalfBath'] + all_data['HalfBath']) * 0.5
all_data['QualSF'] = all_data['TotalSF'] * all_data['OverallQual']
all_data['HasPool'] = all_data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
all_data['HasGarage'] = all_data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
all_data['HasRemod'] = (all_data['YearBuilt'] - all_data['YearRemodAdd']).apply(lambda x: 1 if x != 0 else 0)
all_data['Has3SsnPorch'] = all_data['3SsnPorch'].apply(lambda x: 1 if x > 0 else 0)
all_data['HasLowQualFin'] = all_data['LowQualFinSF'].apply(lambda x: 1 if x > 0 else 0)
all_data['Age'] = all_data['YrSold'] - all_data['YearRemodAdd']
all_data['IsNew'] = (all_data['YrSold'] - all_data['YearBuilt']).apply(lambda x: 1 if x == 0 else 0)
all_data['TotalPorch'] = all_data['OpenPorchSF'] + all_data['EnclosedPorch'] + all_data['3SsnPorch'] + all_data['ScreenPorch']
all_data['HasPorch'] = all_data['TotalPorch'].apply(lambda x: 1 if x > 0 else 0)
all_data['HasFireplace'] = all_data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

# binning
all_data['OpenPorchSF'] = pd.cut(all_data['OpenPorchSF'], [0, 5, 100, 300, 1000], include_lowest=True, labels=False)
all_data['OpenPorchSF'] = all_data['OpenPorchSF'].apply(str)
all_data['EnclosedPorch'] = pd.cut(all_data['EnclosedPorch'], [0, 5, 100, 250, 1000], include_lowest=True, labels=False)
all_data['EnclosedPorch'] = all_data['EnclosedPorch'].apply(str)
all_data['ScreenPorch'] = pd.cut(all_data['ScreenPorch'], [0, 5, 200, 1000], include_lowest=True, labels=False)
all_data['ScreenPorch'] = all_data['ScreenPorch'].apply(str)

In [142]:
# dropping a few columns
to_drop = ['MoSold', 'PoolArea', 'GarageYrBlt', '3SsnPorch', 'YearRemodAdd', 'LowQualFinSF']
all_data.drop(columns=to_drop, inplace=True)
# drop saleprice
all_data.drop(columns='SalePrice', inplace=True)

# redefine categoricals
numerical_features = all_data.dtypes[all_data.dtypes != 'object'].index.values
categorical_features = all_data.dtypes[all_data.dtypes == 'object'].index.values

In [143]:
# transform features based on skewness
skewness_cap = 2 # can change this number around a bit
# define ignored features
ignored_features = ['HasPool', 'HasGarage', 'HasRemod', 'Has3SsnPorch', 'HasLowQualFin', 'IsNew', 'HasPorch']
# find skewness
skewness = all_data[numerical_features].apply(lambda x: st.skew(x)).sort_values(ascending=False)
skewness_features = skewness[abs(skewness) > skewness_cap].index
skewness_features = [f for f in skewness_features if f not in ignored_features]
# box-cox transform
for col in skewness_features:
    all_data[col] = boxcox1p(all_data[col], st.boxcox_normmax(all_data[col] + 1))

# Show result
# adjusted_skewness = all_data[skewness_features].apply(lambda x: st.skew(x))

# skewness_compare = pd.DataFrame()
# skewness_compare['Features'] = skewness_features
# skewness_compare['Original'] = skewness[skewness_features].values
# skewness_compare['Adjusted'] = adjusted_skewness.values
# skewness_compare


In [144]:
# process categoricals
all_data = pd.get_dummies(data=all_data)

In [145]:
# Split up the dataset
train_set = all_data.loc[all_data['Training'] == 1]
test_set = all_data.loc[all_data['Training'] == 0]

# obtain X & y

y_log = np.log1p(y)
target_y = y_log

omit = ['SalePrice', 'Id', 'Training']
X = train_set[[c for c in train_set.columns if c not in omit]]
X_test = test_set[[c for c in test_set.columns if c not in omit]]

# further split into validation set and training set
x_train, x_validation, y_train, y_validation = train_test_split(X, target_y, test_size=0.1, random_state=27)
eval_set = [(x_validation, y_validation)]

In [146]:
# set to True to run all CVs
run_full_tuning = False

In [147]:
# create lasso model and use cv to find best alpha
alphas = np.arange(1e-5, 1e-2, 1e-5)
if run_full_tuning:
    lasso_reg = LassoCV(cv=5, 
        random_state=0, 
        max_iter=50000, 
        alphas=alphas).fit(X, target_y)
    best_alpha = lasso_reg.alpha_ # Lasso alpha: 0.00042

In [148]:
# create ridge model and use cv to find best alpha
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 20, 30, 50, 75, 100]
if run_full_tuning:
    ridge_reg = RidgeCV(cv=5, 
        alphas=ridge_alphas).fit(X, target_y)

    best_ridge_alpha = ridge_reg.alpha_ # Ridge alpha: 10.0

In [149]:
# SVR CV Grid Search
if run_full_tuning:
    svr_model = SVR()

    clf = GridSearchCV(svr_model,
        {'C': [1, 5, 10, 20],
        'epsilon': [0.1, 0.01, 0.001],
        'gamma': [1, 0.1, 0.001, 0.0001, 0.00001]
        }, verbose=1, n_jobs=1)
    clf.fit(X, target_y)

    print(clf.best_score_)
    print(clf.best_params_)
    best_svr_c = clf.best_params_['C']
    best_svr_epsilon = clf.best_params_['epsilon']
    best_svr_gamma = clf.best_params_['gamma']

In [150]:
# grid search for xgb
if False:
    xgb_model = XGBRegressor(n_jobs=1, gamma=0, n_estimators=5000)
    clf = GridSearchCV(xgb_model,
        {'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5, 6]
        }, verbose=1, n_jobs=1)
    clf.fit(X, target_y)

    print(clf.best_score_)
    print(clf.best_params_)
    best_rate_xgb = clf.best_params_['learning_rate']
    best_depth_xgb = clf.best_params_['max_depth']

    xgb_model = XGBRegressor(n_jobs=1, gamma=0, n_estimators=5000, learning_rate=best_rate_xgb, max_depth=best_depth_xgb)
    clf = GridSearchCV(xgb_model,
        {'subsample': [0.5, 0.8, 1],
        'colsample_bytree': [0.5, 0.8, 1]
        }, verbose=1, n_jobs=1)
    clf.fit(X, target_y)

    print(clf.best_score_)
    print(clf.best_params_)
    best_subsample_xgb = clf.best_params_['subsample']
    best_colsample_bytree_xgb = clf.best_params_['colsample_bytree']

    xgb_model = XGBRegressor(n_jobs=1, 
        gamma=0, 
        n_estimators=5000, 
        learning_rate=best_rate_xgb, 
        max_depth=best_depth_xgb,
        subsample=best_subsample_xgb,
        colsample_bytree=best_colsample_bytree_xgb)
    clf = GridSearchCV(xgb_model,
        {'reg_alpha': [0.5, 0.7, 0.9],
        'reg_lambda': [0.5, 0.6, 0.8]
        }, verbose=1, n_jobs=1)
    clf.fit(X, target_y)

    print(clf.best_score_)
    print(clf.best_params_)
    best_alpha_xgb = clf.best_params_['reg_alpha']
    best_lambda_xgb = clf.best_params_['reg_lambda']

In [151]:
# Final Models
lasso = make_pipeline(RobustScaler(), Lasso(random_state=0, 
        max_iter=50000, 
        alpha=0.00042)) 

xgb = XGBRegressor(n_jobs=1, 
    gamma=0, 
    n_estimators=5000, 
    learning_rate=0.01, 
    max_depth=3,
    subsample=0.7,
    reg_alpha=0.1,
    reg_lambda=0.6,
    scale_pos_weight=1,
    min_child_weight=0,
    random_state=27,
    objective='reg:squarederror',
    colsample_bytree=0.7)

ridge = make_pipeline(RobustScaler(), Ridge(alpha=10))

svr = make_pipeline(RobustScaler(), SVR(C=5, epsilon=0.001, gamma=0.0001))

stacked = StackingCVRegressor(regressors=(lasso, xgb, ridge, svr),
    meta_regressor=xgb,
    use_features_in_secondary=True,
    random_state=27)

In [152]:
# Train models for training set and evaluate against the validation set
if run_full_tuning:
    lasso_train = lasso.fit(x_train, y_train)
    ridge_train = ridge.fit(x_train, y_train)
    svr_train = svr.fit(x_train, y_train)
    xgb_train = xgb.fit(x_train, y_train, eval_set=eval_set, early_stopping_rounds=100, eval_metric='rmse')
    stacked_train = stacked.fit(np.array(x_train), np.array(y_train))

In [153]:
# Combining all models
def rmse(y1, y2):
    return np.sqrt(metrics.mean_squared_error(y1, y2))

if run_full_tuning:
    pred_lasso = lasso_train.predict(x_validation)
    pred_xgb = xgb_train.predict(x_validation)
    pred_ridge = ridge_train.predict(x_validation)
    pred_svr = svr_train.predict(x_validation)
    pred_stacked = stacked_train.predict(np.array(x_validation))

    min_error = 100
    best_lasso_weight = 0
    best_xgb_weight = 0
    best_ridge_weight = 0
    best_svr_weight = 0
    best_stacked_weight = 0
    count = 0

    good_weights = pd.DataFrame(columns=['Lasso', 'XGB', 'Ridge', 'SVR', 'Stacked', 'Error'])

    print('Start looking for good weight...')
    for lasso_weight in np.arange(0, 1, 0.05):
        xgb_weights = np.arange(0, 1 - lasso_weight, 0.05)
        for xgb_weight in xgb_weights:
            ridge_weights = np.arange(0, 1 - lasso_weight - xgb_weight, 0.05)
            for ridge_weight in ridge_weights:
                svr_weights = np.arange(0, 1 - lasso_weight - xgb_weight - ridge_weight, 0.05)
                for svr_weight in svr_weights:
                    stacked_weight = 1 - lasso_weight - xgb_weight - ridge_weight - svr_weight
                    
                    count = count + 1
                    y_pred_val = (stacked_weight * pred_stacked + 
                        xgb_weight * pred_xgb + 
                        lasso_weight * pred_lasso + 
                        svr_weight * pred_svr + 
                        ridge_weight * pred_ridge)
                    
                    error = rmse(y_validation, y_pred_val)
                    if error < 0.13: # error cap for a good combination
                        good_weights = good_weights.append({
                            'Lasso': lasso_weight,
                            'XGB': xgb_weight,
                            'Ridge': ridge_weight,
                            'SVR': svr_weight,
                            'Stacked': stacked_weight,
                            'Error': error
                            }, ignore_index=True)
                    if error < min_error:
                        print('Found better weighting at iteration:', count)
                        print(min_error, '->', error)
                        min_error = error
                        best_lasso_weight = lasso_weight
                        best_xgb_weight = xgb_weight
                        best_svr_weight = svr_weight
                        best_ridge_weight = ridge_weight
                        best_stacked_weight = stacked_weight

    print('Best Lasso Weight', best_lasso_weight)
    print('Best XGB Weight', best_xgb_weight)
    print('Best Stacked Weight', best_stacked_weight)
    print('Best Ridge Weight', best_ridge_weight)
    print('Best SVR Weight', best_svr_weight)

    good_weights = good_weights.sort_values('Error')

In [154]:
good_weights.head(20)

,Lasso,XGB,Ridge,SVR,Stacked,Error
8105,0.45,0.15,0.00,0.15,0.25,0.096759
8052,0.45,0.10,0.00,0.15,0.30,0.096768
7762,0.40,0.15,0.00,0.15,0.30,0.096790
7708,0.40,0.10,0.00,0.20,0.30,0.096796
8345,0.50,0.10,0.00,0.15,0.25,0.096806
7807,0.40,0.20,0.00,0.15,0.25,0.096807
7763,0.40,0.15,0.00,0.20,0.25,0.096812
8382,0.50,0.15,0.00,0.10,0.25,0.096817
8143,0.45,0.20,0.00,0.15,0.20,0.096820
8053,0.45,0.10,0.00,0.20,0.25,0.096823


In [159]:
# Train the final model with full dataset
final_lasso = Lasso(random_state=0, 
        max_iter=50000, 
        alpha=0.00042).fit(X, target_y)
final_ridge = Ridge(alpha=10).fit(X, target_y)
final_svr = SVR(C=5, epsilon=0.001, gamma=0.0001).fit(X, target_y)
final_xgb = xgb.fit(x_train, y_train, eval_set=eval_set, early_stopping_rounds=100, eval_metric='rmse')
final_stacked = stacked.fit(np.array(X), np.array(target_y))

149
[905]	validation_0-rmse:0.14147
[906]	validation_0-rmse:0.14148
[907]	validation_0-rmse:0.14149
[908]	validation_0-rmse:0.14149
[909]	validation_0-rmse:0.14149
[910]	validation_0-rmse:0.14149
[911]	validation_0-rmse:0.14151
[912]	validation_0-rmse:0.14149
[913]	validation_0-rmse:0.14148
[914]	validation_0-rmse:0.14148
[915]	validation_0-rmse:0.14149
[916]	validation_0-rmse:0.14148
[917]	validation_0-rmse:0.14151
[918]	validation_0-rmse:0.14151
[919]	validation_0-rmse:0.14150
[920]	validation_0-rmse:0.14149
[921]	validation_0-rmse:0.14145
[922]	validation_0-rmse:0.14143
[923]	validation_0-rmse:0.14142
[924]	validation_0-rmse:0.14141
[925]	validation_0-rmse:0.14140
[926]	validation_0-rmse:0.14132
[927]	validation_0-rmse:0.14133
[928]	validation_0-rmse:0.14130
[929]	validation_0-rmse:0.14128
[930]	validation_0-rmse:0.14128
[931]	validation_0-rmse:0.14128
[932]	validation_0-rmse:0.14118
[933]	validation_0-rmse:0.14112
[934]	validation_0-rmse:0.14114
[935]	validation_0-rmse:0.14116
[936

In [160]:
# prediction result
pred_xgb = final_xgb.predict(X_test)
pred_lasso = final_lasso.predict(X_test)
pred_ridge = final_ridge.predict(X_test)
pred_svr = final_svr.predict(X_test)
pred_stacked = final_stacked.predict(np.array(X_test))

In [162]:
# combine final result and create submission file
y_pred = 0.4 * pred_stacked + 0.4 * pred_lasso + 0.0 * pred_ridge + 0.0 * pred_svr + 0.2 * pred_xgb
y_final_test = np.expm1(y_pred)

# submission
test_data['SalePrice'] = y_final_test
submit = ['SalePrice', 'Id']
submission = test_data[[c for c in test_data.columns if c in submit]]
submission.to_csv('./data/TeamJarvisFinal.csv', index=False)